In [1]:
import numpy as np
import pandas as pd

In [2]:
mom = pd.read_csv('../Data/build_37_dna/Mom_37.csv')

/Users/Dad/.virtualenvs/pandas/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
mom.head()

,RSID,CHROMOSOME,POSITION,RESULT
0,rs4477212,1,82154,AA
1,rs3094315,1,752566,AA
2,rs3131972,1,752721,GG
3,rs12562034,1,768448,--
4,rs12124819,1,776546,GG


In [4]:
martin = pd.read_csv('../Data/build_37_dna/Martin_37.csv')

In [5]:
martin.head()

,RSID,CHROMOSOME,POSITION,RESULT
0,rs4477212,1,82154,AA
1,rs3094315,1,752566,--
2,rs3131972,1,752721,GG
3,rs12562034,1,768448,--
4,rs12124819,1,776546,--


In [10]:
# Notice the my file is a different length due to different version of testing.
len(martin)

720351

In [9]:
len(mom)

720450

In [11]:
merged_left = pd.merge(left=mom,right=martin[['RSID', 'RESULT']], how='left', left_on='RSID', right_on='RSID')

In [12]:
merged_left.head()

,RSID,CHROMOSOME,POSITION,RESULT_x,RESULT_y
0,rs4477212,1,82154,AA,AA
1,rs3094315,1,752566,AA,--
2,rs3131972,1,752721,GG,GG
3,rs12562034,1,768448,--,--
4,rs12124819,1,776546,GG,--


In [13]:
len(merged_left)

720450

In [16]:
merged_left.tail()

,RSID,CHROMOSOME,POSITION,RESULT_x,RESULT_y
720445,rs659139,X,154898561,CC,CC
720446,rs669237,X,154916845,--,--
720447,rs35842692,X,4935714,AA,AC
720448,rs35361563,X,5411068,GG,AG
720449,rs17883004,X,1410495,AG,GG


In [17]:
mom.tail()

,RSID,CHROMOSOME,POSITION,RESULT
720445,rs659139,X,154898561,CC
720446,rs669237,X,154916845,--
720447,rs35842692,X,4935714,AA
720448,rs35361563,X,5411068,GG
720449,rs17883004,X,1410495,AG


In [18]:
# tempory results up until now - not really necessary to save.
merged_left.to_csv('test.csv')

In [19]:
# I know there is a Nan - print to demonstrate:
print(merged_left.loc[[94]])

         RSID CHROMOSOME POSITION RESULT_x RESULT_y
94  rs6685064          1  1211292       CC      NaN


In [20]:
# replace all NaN's with '--':
ml = merged_left.replace(np.nan, '--', regex=True)

In [21]:
print(ml.loc[[94]])

         RSID CHROMOSOME POSITION RESULT_x RESULT_y
94  rs6685064          1  1211292       CC       --


In [22]:
ml.head()

,RSID,CHROMOSOME,POSITION,RESULT_x,RESULT_y
0,rs4477212,1,82154,AA,AA
1,rs3094315,1,752566,AA,--
2,rs3131972,1,752721,GG,GG
3,rs12562034,1,768448,--,--
4,rs12124819,1,776546,GG,--


In [23]:
# Make more meaningful headers:
rml = ml.rename(index=str, columns={"RESULT_x": "Mom", "RESULT_y": "Martin"})

In [24]:
rml.head()

,RSID,CHROMOSOME,POSITION,Mom,Martin
0,rs4477212,1,82154,AA,AA
1,rs3094315,1,752566,AA,--
2,rs3131972,1,752721,GG,GG
3,rs12562034,1,768448,--,--
4,rs12124819,1,776546,GG,--


In [25]:
rml.tail()

,RSID,CHROMOSOME,POSITION,Mom,Martin
720445,rs659139,X,154898561,CC,CC
720446,rs669237,X,154916845,--,--
720447,rs35842692,X,4935714,AA,AC
720448,rs35361563,X,5411068,GG,AG
720449,rs17883004,X,1410495,AG,GG


In [26]:
# Brute force method of figuring out every Allele given to me by my mother:
# Doubling the results allow me to compare the results to other data down the road.
# There are likely more elegant ways to do this.
# If there are any mistakes please let me know or update it.
def get_common_allele (row):
    # (<Mom>, <Martin>):<Common>,
    mom2Child = { ("AA", "AA"):"AA",
                  ("AA", "AC"):"AA",
                  ("AA", "AG"):"AA",
                  ("AA", "AT"):"AA",
                  ("AA", "CC"):"AA",
                  ("AA", "GG"):"AA",
                  ("AA", "--"):"AA",
                  ("AC", "AA"):"AA",
                  ("AC", "AC"):"--",
                  ("AC", "CC"):"CC",
                  ("AC", "--"):"--",
                  ("AG", "AA"):"AA",
                  ("AG", "AG"):"--",
                  ("AG", "GG"):"GG",
                  ("AG", "--"):"--",
                  ("AT", "AA"):"AA",
                  ("AT", "AT"):"--",
                  ("AT", "TT"):"TT",
                  ("AT", "--"):"--",
                  ("CC", "AA"):"CC",
                  ("CC", "AC"):"CC",
                  ("CC", "CC"):"CC",
                  ("CC", "CG"):"CC",
                  ("CC", "CT"):"CC",
                  ("CC", "TT"):"CC",
                  ("CC", "--"):"CC",
                  ("CC", "GG"):"CC",
                  ("CG", "CC"):"CC",
                  ("CG", "CG"):"--",
                  ("CG", "GG"):"CC",
                  ("CG", "--"):"--",
                  ("CT", "CC"):"CC",
                  ("CT", "CT"):"--",
                  ("CT", "TT"):"TT",
                  ("CT", "--"):"--",
                  ("GG", "AA"):"GG",
                  ("GG", "AG"):"GG",
                  ("GG", "CG"):"GG",
                  ("GG", "GG"):"GG",
                  ("GG", "GT"):"GG",
                  ("GG", "TT"):"GG",
                  ("GG", "--"):"GG",
                  ("GT", "GG"):"GG",
                  ("GT", "GT"):"--",
                  ("GT", "TT"):"TT",
                  ("GT", "--"):"--",
                  ("TT", "AT"):"TT",
                  ("TT", "CC"):"TT",
                  ("TT", "CT"):"TT",
                  ("TT", "GG"):"TT",
                  ("TT", "GT"):"TT",
                  ("TT", "TT"):"TT",
                  ("TT", "--"):"TT",
                  ("--", "AA"):"AA",
                  ("--", "AC"):"--",
                  ("--", "AG"):"--",
                  ("--", "AT"):"--",
                  ("--", "CC"):"CC",
                  ("--", "CG"):"--",
                  ("--", "CT"):"--",
                  ("--", "GG"):"GG",
                  ("--", "GT"):"--",
                  ("--", "TT"):"TT",
                  ("--", "--"):"--",
                  ("RESULT", "RESULT"):"RESULT"
                }
    return mom2Child[(row['Mom'], row['Martin'])]

In [27]:
rml['RESULT'] = rml.apply (lambda row: get_common_allele (row),axis=1)

In [28]:
rml.head()

,RSID,CHROMOSOME,POSITION,Mom,Martin,RESULT
0,rs4477212,1,82154,AA,AA,AA
1,rs3094315,1,752566,AA,--,AA
2,rs3131972,1,752721,GG,GG,GG
3,rs12562034,1,768448,--,--,--
4,rs12124819,1,776546,GG,--,GG


In [29]:
rml.tail()

,RSID,CHROMOSOME,POSITION,Mom,Martin,RESULT
720445,rs659139,X,154898561,CC,CC,CC
720446,rs669237,X,154916845,--,--,--
720447,rs35842692,X,4935714,AA,AC,AA
720448,rs35361563,X,5411068,GG,AG,GG
720449,rs17883004,X,1410495,AG,GG,GG


In [30]:
# Create a mono DNA file:
MomAndMartin = rml.drop(['Mom', 'Martin'], axis=1)

In [31]:
MomAndMartin.head()

,RSID,CHROMOSOME,POSITION,RESULT
0,rs4477212,1,82154,AA
1,rs3094315,1,752566,AA
2,rs3131972,1,752721,GG
3,rs12562034,1,768448,--
4,rs12124819,1,776546,GG


In [34]:
# Save results in a csv with index removed:
MomAndMartin.to_csv('MomAndMartin.csv.gz', compression='gzip', index=False)

In [35]:
# Using a similar method, derive what is know an the evil twin :
# An evil twin is an imaginary sibling of mine who received the other allele from my mother at every snp.
# For example, if my mother is AA and I am AC at a snp, I received an A from her, my evil twin would have received a C.
def get_other_allele (row):
    # (<Mom>, <Martin>):<EvilTwin>,
    mom2EvilTwin = { ("AA", "AA"):"AA",
                  ("AA", "AC"):"AA",
                  ("AA", "AG"):"AA",
                  ("AA", "AT"):"AA",
                  ("AA", "CC"):"AA",
                  ("AA", "GG"):"AA",
                  ("AA", "--"):"AA",
                  ("AC", "AA"):"CC",
                  ("AC", "AC"):"--",
                  ("AC", "CC"):"AA",
                  ("AC", "--"):"--",
                  ("AG", "AA"):"GG",
                  ("AG", "AG"):"--",
                  ("AG", "GG"):"AA",
                  ("AG", "--"):"--",
                  ("AT", "AA"):"TT",
                  ("AT", "AT"):"--",
                  ("AT", "TT"):"AA",
                  ("AT", "--"):"--",
                  ("CC", "AA"):"CC",
                  ("CC", "AC"):"CC",
                  ("CC", "CC"):"CC",
                  ("CC", "CG"):"CC",
                  ("CC", "CT"):"CC",
                  ("CC", "TT"):"CC",
                  ("CC", "--"):"CC",
                  ("CC", "GG"):"CC",
                  ("CG", "CC"):"GG",
                  ("CG", "CG"):"--",
                  ("CG", "GG"):"CC",
                  ("CG", "--"):"--",
                  ("CT", "CC"):"TT",
                  ("CT", "CT"):"--",
                  ("CT", "TT"):"CC",
                  ("CT", "--"):"--",
                  ("GG", "AA"):"GG",
                  ("GG", "AG"):"GG",
                  ("GG", "CG"):"GG",
                  ("GG", "GG"):"GG",
                  ("GG", "GT"):"GG",
                  ("GG", "TT"):"GG",
                  ("GG", "--"):"GG",
                  ("GT", "GG"):"TT",
                  ("GT", "GT"):"--",
                  ("GT", "TT"):"GG",
                  ("GT", "--"):"--",
                  ("TT", "AT"):"TT",
                  ("TT", "CC"):"TT",
                  ("TT", "CT"):"TT",
                  ("TT", "GG"):"TT",
                  ("TT", "GT"):"TT",
                  ("TT", "TT"):"TT",
                  ("TT", "--"):"TT",
                  ("--", "AA"):"--",
                  ("--", "AC"):"--",
                  ("--", "AG"):"--",
                  ("--", "AT"):"--",
                  ("--", "CC"):"--",
                  ("--", "CG"):"--",
                  ("--", "CT"):"--",
                  ("--", "GG"):"--",
                  ("--", "GT"):"--",
                  ("--", "TT"):"--",
                  ("--", "--"):"--",
                  ("RESULT", "RESULT"):"RESULT"
                }
    return mom2EvilTwin[(row['Mom'], row['Martin'])]

In [36]:
rml['RESULT'] = rml.apply (lambda row: get_other_allele (row),axis=1)

In [37]:
rml.head()

,RSID,CHROMOSOME,POSITION,Mom,Martin,RESULT
0,rs4477212,1,82154,AA,AA,AA
1,rs3094315,1,752566,AA,--,AA
2,rs3131972,1,752721,GG,GG,GG
3,rs12562034,1,768448,--,--,--
4,rs12124819,1,776546,GG,--,GG


In [40]:
# Only keep reults row:
MartinsEvilTwin = rml.drop(['Mom', 'Martin'], axis=1)

In [41]:
MartinsEvilTwin.head()

,RSID,CHROMOSOME,POSITION,RESULT
0,rs4477212,1,82154,AA
1,rs3094315,1,752566,AA
2,rs3131972,1,752721,GG
3,rs12562034,1,768448,--
4,rs12124819,1,776546,GG


In [42]:
MartinsEvilTwin.to_csv('MartinsEvilTwin.csv.gz', compression='gzip', index=False)

In [44]:
# Here is how I would determine the Allele I received from my father, based on what I received from my mother:
# I don't double the Allele here because I will determine my dad's DNA from my siblings' DNA as well, then get a 
# more accurate estimate based on all of our DNA. This is here, for now, simply as a demonstration.
def get_dads_allele (row):
    # (<Mom>, <Martin>):<Dad>,
    dadFromMom= { ("AA", "AA"):"A",
                  ("AA", "AC"):"C",
                  ("AA", "AG"):"G",
                  ("AA", "AT"):"T",
                  ("AA", "CC"):"-",
                  ("AA", "GG"):"-",
                  ("AA", "--"):"-",
                  ("AC", "AA"):"A",
                  ("AC", "AC"):"-",
                  ("AC", "CC"):"C",
                  ("AC", "--"):"-",
                  ("AG", "AA"):"A",
                  ("AG", "AG"):"-",
                  ("AG", "GG"):"G",
                  ("AG", "--"):"-",
                  ("AT", "AA"):"A",
                  ("AT", "AT"):"-",
                  ("AT", "TT"):"T",
                  ("AT", "--"):"-",
                  ("CC", "AA"):"-",
                  ("CC", "AC"):"A",
                  ("CC", "CC"):"C",
                  ("CC", "CG"):"G",
                  ("CC", "CT"):"T",
                  ("CC", "TT"):"-",
                  ("CC", "--"):"-",
                  ("CC", "GG"):"-",
                  ("CG", "CC"):"C",
                  ("CG", "CG"):"-",
                  ("CG", "GG"):"-",
                  ("CG", "--"):"-",
                  ("CT", "CC"):"C",
                  ("CT", "CT"):"-",
                  ("CT", "TT"):"T",
                  ("CT", "--"):"-",
                  ("GG", "AA"):"-",
                  ("GG", "AG"):"A",
                  ("GG", "CG"):"C",
                  ("GG", "GG"):"G",
                  ("GG", "GT"):"T",
                  ("GG", "TT"):"-",
                  ("GG", "--"):"-",
                  ("GT", "GG"):"G",
                  ("GT", "GT"):"-",
                  ("GT", "TT"):"T",
                  ("GT", "--"):"-",
                  ("TT", "AT"):"A",
                  ("TT", "CC"):"_",
                  ("TT", "CT"):"C",
                  ("TT", "GG"):"-",
                  ("TT", "GT"):"G",
                  ("TT", "TT"):"T",
                  ("TT", "--"):"-",
                  ("--", "AA"):"-",
                  ("--", "AC"):"-",
                  ("--", "AG"):"-",
                  ("--", "AT"):"-",
                  ("--", "CC"):"-",
                  ("--", "CG"):"-",
                  ("--", "CT"):"-",
                  ("--", "GG"):"-",
                  ("--", "GT"):"-",
                  ("--", "TT"):"-",
                  ("--", "--"):"-",
                  ("RESULT", "RESULT"):"RESULT"
                }
    return dadFromMom[(row['Mom'], row['Martin'])]

In [46]:
rml['RESULT'] = rml.apply (lambda row: get_dads_allele (row),axis=1)

In [47]:
rml

,RSID,CHROMOSOME,POSITION,Mom,Martin,RESULT
0,rs4477212,1,82154,AA,AA,A
1,rs3094315,1,752566,AA,--,-
2,rs3131972,1,752721,GG,GG,G
3,rs12562034,1,768448,--,--,-
4,rs12124819,1,776546,GG,--,-
5,rs11240777,1,798959,GG,AG,A
6,rs6681049,1,800007,CC,--,-
7,rs4970383,1,838555,CC,CC,C
8,rs4475691,1,846808,CC,CC,C
9,rs7537756,1,854250,AA,AA,A
